In [1]:
import os
import torch
import torch.nn as nn
import pandas as pd
import torch.nn.functional as F
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.datasets import CelebA
from PIL import Image
from sklearn.metrics import accuracy_score
from facenet_pytorch import InceptionResnetV1
from kd_framework import dataset, kd_models, mobilefacenet, utils

c:\Users\Public\anaconda3\envs\YOUCEF_OUADJER_ENV\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(os.getcwd())

e:\youcef ouadjer


In [3]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [4]:

# Charger le fichier identity et le fichier de split
identity_df = pd.read_csv('celeba/identity_CelebA.txt', sep=' ', header=None, names=['filename', 'identity'])
partition_df = pd.read_csv('celeba/list_eval_partition.txt', sep=' ', header=None, names=['filename', 'split'])

# Fusionner pour avoir split + identity
full_df = pd.merge(identity_df, partition_df, on='filename')

# Mapping pour chaque split
split_names = {0: 'train', 1: 'valid', 2: 'test'}
split_mappings = {}

for split_id, split_name in split_names.items():
    split_subset = full_df[full_df['split'] == split_id]
    unique_ids = sorted(split_subset['identity'].unique())
    id2label = {int(identity): i for i, identity in enumerate(unique_ids)}
    split_mappings[split_name] = id2label
    print(f"[{split_name}] → {len(id2label)} unique identities")

# Exemple d'utilisation pour `val_dataset`
val_id2label = split_mappings['valid']


[train] → 8192 unique identities
[valid] → 985 unique identities
[test] → 1000 unique identities


In [5]:
unique_ids = sorted(full_df['identity'].unique())
id2label_global = {int(identity): i for i, identity in enumerate(unique_ids)}

In [6]:
len(id2label_global)

10177

In [7]:
# Dataset 

transform_teacher = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

transform_student = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Mapping global
#id2label = {int(identity): idx for idx, identity in enumerate(unique_ids)}

# Créer les datasets avec le même mapping

# train_dataset = dataset.celebATransformDataset('train', transform_teacher, transform_student, id2label=split_mappings['train'])
# val_dataset   = dataset.celebATransformDataset('valid', transform_teacher, transform_student, id2label=split_mappings['valid'])
# test_dataset  = dataset.celebATransformDataset('test',  transform_teacher, transform_student, id2label=split_mappings['test'])

train_dataset = dataset.celebATransformDataset('train', transform_teacher, transform_student, id2label=id2label_global)
val_dataset   = dataset.celebATransformDataset('valid', transform_teacher, transform_student, id2label=id2label_global)
test_dataset  = dataset.celebATransformDataset('test',  transform_teacher, transform_student, id2label=id2label_global)


train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False, num_workers=4)

In [8]:
for img_teacher, img_student, label in train_loader:
    print(img_teacher.size())
    print(img_student.size())
    print(label.size())
    break

torch.Size([512, 3, 160, 160])
torch.Size([512, 3, 112, 112])
torch.Size([512])


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
# Models
# Teacher model: FaceNet (InceptionResNet)
#num_train_classes = len(split_mappings['train'])

num_train_classes = len(id2label_global)
resnet_model = InceptionResnetV1(pretrained='vggface2').eval()
teacher_model = kd_models.TeacherNet(resnet_model, num_classes=num_train_classes).to(device)

In [11]:
# Student model: MobileFaceNet
#num_val_classes = len(split_mappings['valid'])

num_val_classes = len(id2label_global)

mobile_model = mobilefacenet.MobileFaceNet(embedding_size=512).to(device)
student_model = kd_models.StudentNet(mobile_model, num_classes=num_val_classes).to(device)

In [12]:
x_teacher  = torch.randn(32, 3, 160, 160).to(device)
x_student = torch.randn(32, 3, 112, 112).to(device)

In [13]:
t_logits, t_feats = teacher_model(x_teacher)
s_logits, s_feats, embeddings = student_model(x_student)

In [14]:
print(t_logits.shape)
# print(t_feats.shape)

print(s_logits.shape)
# print(s_feats.shape)
print(embeddings.shape)

torch.Size([32, 10177])
torch.Size([32, 10177])
torch.Size([32, 512])


In [15]:
# Training + Evaluation loop
optimizer = torch.optim.Adam(student_model.parameters(), lr=0.001)
epochs = 500

save_dir = "./checkpoints"
os.makedirs(save_dir, exist_ok=True)

best_val_acc = 0  # For tracking the best model

for epoch in tqdm(range(epochs)):
    # ------------------ Training ------------------
    student_model.train()
    total_loss, total_correct, total_samples = 0, 0, 0

    for img_teacher, img_student, labels in train_loader:
        img_teacher = img_teacher.cuda()
        img_student = img_student.cuda()
        labels = labels.cuda()

        with torch.no_grad():
            teacher_logits, teacher_feats = teacher_model(img_teacher)

        student_logits, student_feats, _ = student_model(img_student)
        
        # print(f"Label range: min={labels.min().item()}, max={labels.max().item()}")
        # print(f"Student logits shape: {student_logits.shape}")

        #print(f"Teacher logits shape: {teacher_logits.shape}")
        #print(f"Student labels shape: {labels.shape}")

        loss = utils.distillation_loss(student_logits, teacher_logits, student_feats, teacher_feats, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * labels.size(0)
        total_correct += (student_logits.argmax(1) == labels).sum().item()
        total_samples += labels.size(0)

    train_acc = 100 * total_correct / total_samples
    avg_loss = total_loss / total_samples
    print(f"Epoch {epoch+1}: Train Loss={avg_loss:.4f}, Train Acc={train_acc:.2f}%")

    # ------------------ Evaluation ------------------
    student_model.eval()
    all_preds = []
    all_labels = []
    total_val_loss = 0

    with torch.no_grad():
        for img_teacher, img_student, labels in val_loader:
            
            img_student = img_student.cuda()
            labels = labels.cuda()

            student_logits, _, _ = student_model(img_student)
            #print(f"Student labels shape: {labels.shape}")
            preds = student_logits.argmax(1)

            # print(f"Label range: min={labels.min().item()}, max={labels.max().item()}")
            #print(f"Student logits shape: {student_logits.shape}")
            
            #print(f"Predictions shape: {preds.shape}")
            #print(f"labels shape: {labels.shape}")
            # print(f"Predictions range: min={preds.min().item()}, max={preds.max().item()}")


            val_loss = F.cross_entropy(student_logits, labels)
            total_val_loss += val_loss.item() * labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_loss_avg = total_val_loss / len(val_loader.dataset)
    val_acc = accuracy_score(all_labels, all_preds)
    print(f"Val Loss={val_loss_avg:.4f}, Val Acc={val_acc * 100:.2f}%\n")

    # Save model after every epoch
    torch.save(student_model.state_dict(), os.path.join(save_dir, f"student_epoch_{epoch+1}.pth"))

    # Or, save only the best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(student_model.state_dict(), os.path.join(save_dir, "best_student_model.pth"))
        print(f"Best model saved at epoch {epoch+1} with Val Acc={val_acc * 100:.2f}%")


  0%|          | 0/500 [00:00<?, ?it/s]

Epoch 1: Train Loss=263.6340, Train Acc=1.28%


  0%|          | 1/500 [03:27<28:48:43, 207.86s/it]

Val Loss=9.7903, Val Acc=0.00%

Epoch 2: Train Loss=3.8303, Train Acc=10.79%


  0%|          | 2/500 [06:55<28:42:26, 207.52s/it]

...

Val Loss=9.7924, Val Acc=0.00%

Epoch 215: Train Loss=0.3380, Train Acc=99.84%


 43%|████▎     | 215/500 [12:46:50<22:31:05, 284.44s/it]

Val Loss=9.7923, Val Acc=0.00%



 43%|████▎     | 215/500 [12:50:50<17:01:48, 215.12s/it]


KeyboardInterrupt: 